# HITS
Ahora vamos a practicar el algoritmo HITS al igual que usamos PageRank para el mismo ejemplo de grafo

## Parte 2 - Hoja de trabajo
### Introducción


El algoritmo HITS se basa en asignar a las páginas un valor de autoridad y un valor de hub (página que apunta a páginas con autoridad).


In [1]:
# Antes de empezar, carguemos los paquetes necesarios
%matplotlib inline
import numpy as np
import numpy.linalg as la
import scipy.io as sio
from math import isclose
from pathlib import Path
import urllib.request
import os
np.set_printoptions(suppress=True)

Podemos cargar la matriz de adyacencia de un subconjunto de la Wikipedia usando el método `loadmat`
 que nos devolverá una matriz dispersa. Podemos comprobar que es dispersa simplemente pidiendo el tamaño que tiene.

In [2]:
cwd = os.getcwd()
fichero_matriz = 'wiki-adj-matrix.mat'
fichero_nodos = 'wiki-nodenames.txt'
wikimatfile = Path(cwd)/fichero_matriz
wikinodesfile = Path(cwd)/fichero_nodos

if not wikimatfile.exists():
    urllib.request.urlretrieve('https://ditec.um.es/~lfmaimo/docencia/ri/{}'.format(fichero_matriz), wikimatfile)
    print('Matriz descargada en {}...\n'.format(cwd))

if not wikinodesfile.exists():
    urllib.request.urlretrieve('https://ditec.um.es/~lfmaimo/docencia/ri/{}'.format(fichero_nodos), wikinodesfile)
    print('Nodos descargados en {}...\n'.format(cwd))


In [3]:
wikipedia = sio.loadmat("wiki-adj-matrix.mat")  # Cargamos la matriz de adyacencia de las páginas de wikipedia
A = wikipedia['A']
print(f"Dimensiones: {A.shape}  Número de elementos distintos de cero: {A.size}") 
with open("wiki-nodenames.txt","r") as wiki_names_file:
    wiki_names = wiki_names_file.read().splitlines()

Dimensiones: (3357835, 3357835)  Número de elementos distintos de cero: 40390894


In [4]:
n = A.shape[0]

In [5]:
a = np.ones(n)
h = np.ones(n)

Se pide que calculéis los vectores a y h del algoritmo HITS tal y como se ha visto en clase. Deberéis utilizar el método iterativo donde se calcula inicialmente $\vec{a} = A^T\vec{h}$, se normaliza el resultado, y se pasa a calcular $\vec{h} = A\vec{a}$, se normaliza de nuevo el resultado y se itera hasta que la norma de la diferencia de dos vectores consecutivos sea menor que un cierto épsilon.

In [6]:
eps = 0.001
# BEGIN YOUR CODE
max_iter = 1000

for i in range(max_iter):
    # Calcular autoridades: a = A^T @ h
    a_new = A.T @ h
    # Normalizar
    a_new = a_new / la.norm(a_new)
    
    # Calcular hubs: h = A @ a_new
    h_new = A @ a_new
    # Normalizar
    h_new = h_new / la.norm(h_new)
    
    # Verificar convergencia
    if la.norm(a_new - a) < eps:
        a = a_new
        h = h_new
        break
    
    # Actualizar para la siguiente iteración
    a = a_new
    h = h_new
# END YOUR CODE

In [7]:
ind_max_a = np.argsort(a)[-4:][::-1]
ind_max_h = np.argsort(h)[-4:][::-1]
print(f"4 mayores valores de a = {a[ind_max_a]}, correspondiendo a los términos: {[wiki_names[i] for i in ind_max_a]}")   
print(f"4 mayores valores de h = {h[ind_max_h]}, correspondiendo a los términos: {[wiki_names[i] for i in ind_max_h]}")   


4 mayores valores de a = [0.00784568 0.00779136 0.00776166 0.00747651], correspondiendo a los términos: ['Savannah, Georgia', 'Lima, Ohio', 'List of disasters', 'El Paso, Texas']
4 mayores valores de h = [0.37751002 0.24842314 0.24705197 0.2448069 ], correspondiendo a los términos: ['United States', 'Population density', 'Census', 'Marriage']


La salida debería ser la siguiente:

Podríamos calcular directamente a y h usando el método de la potencia sobre $A^T A$ y $A A^T$ respectivamente. Pero si intentas calcular esas matrices directamente, resulta que es más costoso que aplicar el método directamente. Prueba el tiempo que tarda en calcularlas ejecutando las siguientes celdas. 

NOTA: Si tarda demasiado, puede que tu kernel sea reseteado, así que comenta las celdas y no las ejecutes. No son necesarias para el resto del notebook.

In [8]:
# %%time
# aat = A @ A.T

In [9]:
# %%time
# ata = A.T @ A

Además, aunque la matriz A sea dispersa, el producto de A por su traspuesta en general no tiene por qué ser tan disperso. Así que podemos ahorrarnos todo el tiempo de la multiplicación anterior, simplemente haciendo el producto $A^T A a$ o $A A^T h$ por partes. Utiliza las expresiones anteriores para, poniendo los paréntesis adecuados, calcular los vectores $\vec{a}$ y $\vec{h}$.

In [10]:
eps = 0.001
# BEGIN YOUR CODE
a = np.ones(n)
max_iter = 1000

for i in range(max_iter):
    # Calcular (A^T A) a de forma eficiente: A^T @ (A @ a)
    a_new = A.T @ (A @ a)
    # Normalizar
    a_new = a_new / la.norm(a_new)
    
    # Verificar convergencia
    if la.norm(a_new - a) < eps:
        a = a_new
        break
    
    a = a_new

# END YOUR CODE
ind_max_a = np.argsort(a)[-4:][::-1]
print(f"4 mayores valores de a = {a[ind_max_a]}, correspondiendo a los términos: {[wiki_names[i] for i in ind_max_a]}")   


4 mayores valores de a = [0.00784324 0.00778931 0.00775053 0.00747486], correspondiendo a los términos: ['Savannah, Georgia', 'Lima, Ohio', 'List of disasters', 'El Paso, Texas']


La salida debería ser la siguiente.

In [11]:
eps = 0.001
# BEGIN YOUR CODE

h = np.ones(n)
max_iter = 1000

for i in range(max_iter):
    # Calcular (A A^T) h de forma eficiente: A @ (A^T @ h)
    h_new = A @ (A.T @ h)
    # Normalizar
    h_new = h_new / la.norm(h_new)
    
    # Verificar convergencia
    if la.norm(h_new - h) < eps:
        h = h_new
        break
    
    h = h_new

# END YOUR CODE
ind_max_h = np.argsort(h)[-4:][::-1]
print(f"4 mayores valores de h = {h[ind_max_h]}, correspondiendo a los términos: {[wiki_names[i] for i in ind_max_h]}")   


4 mayores valores de h = [0.37751002 0.24842314 0.24705197 0.2448069 ], correspondiendo a los términos: ['United States', 'Population density', 'Census', 'Marriage']


Con la siguiente salida:

# Usos de Pagerank en otros contextos

Para demostrar que Pagerank es un algoritmo que puede aportar información sobre grafos diferentes de páginas webs, vamos a utilizar la implementación de Pagerank densa que hicisteis en el anterior notebook para determinar qué equipo es el mejor en una liga de partidos de fútbol americano. Para ello construiremos la matriz de adyacencia de un grafo, donde cada equipo corresponde con un nodo diferente y un nodo $i$ tendrá un arco a un nodo $j$ si el equipo correspondiente al nodo $i$ ha perdido frente al equipo correspondiente al nodo $j$.

El dataset de equipos y partidos se encuentra en los ficheros ``equipos.txt`` y ``partidos.txt`` que os proporcionamos en la sección de recursos. Tendréis que leer dichos ficheros y crear la matriz de adyacencia a partir de la información que se encuentra en ellos, que se explica a continuación.

- ``equipos.txt`` contiene la lista de los equipos separada por comas.
- ``partidos.txt`` contiene la lista de los partidos, uno por línea, en el siguiente formato: Equipo_1, puntuación_1, donde_juegan (este campo no importa), equipo_2, puntuación_2. Ejemplo: Arkansas,41,at,Texas Christian,38 indicaría que Arkansas obtuvo 41 puntos jugando contra Texas Christian, que obtuvo 38 puntos.

Obtén los 4 mejores equipos de la liga con los correspondientes Pageranks. No hagas ninguna alteración de la matriz de adyacencia, ya que el grafo de partidos no tiene bucles (un equipo no juega contra sí mismo) ni hay sumideros, ya que en una liga todos juegan con alguien alguna vez. Toma como factor de amortiguación $\lambda = 0.85$.

In [12]:
# Del notebook anterior inserta aquí la función pagerank_dense

def pagerank_dense(Ld, d=0.5, eps=0.00001):
    """ Aplica el método de la potencia como hemos implementado anteriormente
        hasta que la diferencia entre la norma de dos vectores consecutivos sea menor que eps,
        o que dicha diferencia no disminuya, sino que aumente, en cuyo caso se considerará que
        el método no converge.
        
        Args:
            Ld: Matriz de transición densa (numpy array)
            d: factor de amortiguamiento (float)
            eps: tolerancia para la convergencia (float)

        Devuelve el vector resultante y el número de iteraciones. 
        Si no converge, el número de iteraciones devuelto será 0
    """
    length = Ld.shape[0]
    r = np.ones(length)/length
    i = 0
    # BEGIN YOUR CODE
    prev_diff = float('inf') # Diferencia previa, inicialmente infinita

    while True:
        # Calcular la nueva matriz M para esta iteración
        n = length
        J = np.ones([n, n])
        M = d * Ld + (1 - d) / n * J

        # Calcular r^(i+1) = M * r^(i)
        r_new = M @ r

        # Calcular la norma de la diferencia
        diff = np.linalg.norm(r_new - r)

        # Comprobar si la diferencia es menor que eps
        if diff < eps:
            r = r_new
            i += 1
            break

        # Comprobar si la diferencia ha aumentado (no converge)
        if diff > prev_diff:
            # No converge
            return r, 0 # Devuelve vector actual y 0 iteraciones

        # Actualizar r, diferencia previa e iterador
        r = r_new
        prev_diff = diff
        i += 1
    # END YOUR CODE
    return r, i 

In [13]:
# BEGIN YOUR CODE
# Usa el número de celdas que necesites

# 1. Leer equipos
with open("equipos.txt", "r") as f:
    equipos = f.read().strip().split(",")

# Crear diccionarios de índices para los equipos
n_equipos = len(equipos)
equipo_idx = {equipo: i for i, equipo in enumerate(equipos)}

# 2. Crear matrices de adyacencia (inicialmente con ceros)
adj_matrix = np.zeros((n_equipos, n_equipos))

# 3. Leer partidos y llenar la matriz
with open("partidos.txt", "r") as f:
    for line in f:
        parts = line.strip().split(',')
        equipo1 = parts[0]
        puntos1 = int(parts[1])
        equipo2 = parts[3]
        puntos2 = int(parts[4])

        # El perdedor apunta al ganador (arco del perdedor al ganador)
        idx1 = equipo_idx[equipo1]
        idx2 = equipo_idx[equipo2]

        if puntos1 > puntos2:
            # Equipo 2 perdió, apunta a equipo 1
            adj_matrix[idx2, idx1] = 1
        elif puntos2 > puntos1:
            # Equipo 1 perdió, apunta a equipo 2
            adj_matrix[idx1, idx2] = 1

# 4. Crear matriz de transición (normalizar columnas)
# Calcular el out-degree de cada nodo (suma por filas)
out_degree = adj_matrix.sum(axis=1)

# Crear matriz de transición L
L = np.zeros((n_equipos, n_equipos))
for i in range(n_equipos):
    if out_degree[i] > 0:
        L[:, i] = adj_matrix[i, :] / out_degree[i]

# 5. Aplicar PageRank con d=0.85
pg, iters = pagerank_dense(L, d=0.85)

# 6. Obtener los 4 mejores equipos
ind_max_pg = np.argsort(pg)[-4:][::-1]
print(f"4 mayores valores de pg = {pg[ind_max_pg]}, correspondiendo a los términos: {[equipos[i] for i in ind_max_pg]}")
# END YOUR CODE

4 mayores valores de pg = [0.06215507 0.05840507 0.02792401 0.024991  ], correspondiendo a los términos: ['Pittsburgh', 'Clemson', 'Alabama', 'Ohio State']


Los resultados deberían ser los siguientes.

Compara el resultado anterior con la siguiente modificación a los pesos de la matriz de adyacencia. Como no hemos tenido en cuenta los resultados de los partidos, se podría hacer una modificación en los arcos, dándole a un arco de la matriz de adyacencia un peso igual a la diferencia de puntos por la que ha sido vencido un equipo. Repite los pasos anteriores para ver cuál sería el resultado con esta nueva configuración. Usa el mismo valor de amortiguación $\lambda = 0.85$.

In [14]:
# BEGIN YOUR CODE
# Usa el número de celdas que necesites
# 1. Leer equipos
with open("equipos.txt", "r") as f:
    equipos = f.read().strip().split(",")

# Crear diccionarios de índices para los equipos
n_equipos = len(equipos)
equipo_idx = {equipo: i for i, equipo in enumerate(equipos)}

# 2. Crear matrices de adyacencia (inicialmente con ceros)
adj_matrix_weighted = np.zeros((n_equipos, n_equipos))

# 3. Leer partidos y llenar la matriz
with open("partidos.txt", "r") as f:
    for line in f:
        parts = line.strip().split(',')
        equipo1 = parts[0]
        puntos1 = int(parts[1])
        equipo2 = parts[3]
        puntos2 = int(parts[4])

        # El perdedor apunta al ganador con peso = diferencia de puntos
        idx1 = equipo_idx[equipo1]
        idx2 = equipo_idx[equipo2]

        if puntos1 > puntos2:
            adj_matrix_weighted[idx2, idx1] = puntos1 - puntos2
        elif puntos2 > puntos1:
            adj_matrix_weighted[idx1, idx2] = puntos2 - puntos1

# 4. Crear matriz de transición (normalizar columnas)
# Calcular el out-degree de cada nodo (suma por filas)
out_degree_weighted = adj_matrix_weighted.sum(axis=1)

# Crear matriz de transición L
L_weighted = np.zeros((n_equipos, n_equipos))
for i in range(n_equipos):
    if out_degree[i] > 0:
        L_weighted[:, i] = adj_matrix_weighted[i, :] / out_degree_weighted[i]

# 5. Aplicar PageRank con d=0.85
pg_weighted, iters = pagerank_dense(L_weighted, d=0.85)

# 6. Obtener los 4 mejores equipos
ind_max_pg_weighted = np.argsort(pg_weighted)[-4:][::-1]
print(f"4 mayores valores de pg = {pg_weighted[ind_max_pg_weighted]}, correspondiendo a los términos: {[equipos[i] for i in ind_max_pg_weighted]}")
# END YOUR CODE

4 mayores valores de pg = [0.08683448 0.07791288 0.04632329 0.04610838], correspondiendo a los términos: ['Clemson', 'Pittsburgh', 'Alabama', 'Miami (Florida)']


El resultado debería ser el siguiente.